# 1. CRAWLER

In [32]:
# crawl code

import requests
from bs4 import BeautifulSoup
from openpyxl import Workbook

def get_last_page_num(keyword,term):
    max = 500
    page_num = 1
    try:
        for i in range(1, max * 10, 10):
            response = requests.get(f"https://search.naver.com/search.naver?where=news&sm=tab_jum&query={keyword}&start={i}&pd={term}",headers={'User-Agent':'Mozilla/5.0'})
            html = response.text
            soup = BeautifulSoup(html, "html.parser")
            btn_next = soup.select_one("a.btn_next")
            if(btn_next == None):
                return 0
            is_last_page = btn_next.attrs["aria-disabled"]
            if is_last_page == "true":
                break

            page_num += 1
    except:
        pass
    return page_num


def crawler(keyword, lastpage, term):
    wb = Workbook()
    if(keyword=="다."):
        total_sheet = wb.create_sheet("전체")
    else:
        total_sheet = wb.create_sheet(keyword)
    total_sheet.column_dimensions['A'].width = 10
    total_sheet.column_dimensions['B'].width = 60
    total_sheet.column_dimensions['C'].width = 60
    total_sheet.column_dimensions['D'].width = 150

    row = 0
    for i in range(1, lastpage * 10, 10):
        try:
            response = requests.get(f"https://search.naver.com/search.naver?where=news&sm=tab_jum&query={keyword}&start={i}&pd={term}",headers={'User-Agent':'Mozilla/5.0'})
            html = response.text
            soup = BeautifulSoup(html, "html.parser")
            articles = soup.select("div.news_wrap.api_ani_send")

            for article in articles:
                title = ""
                content = ""

                info_group = article.select_one("div.info_group")
                links = info_group.select("a.info")
                if len(links) >= 2:
                    url = links[1].attrs["href"]
                    if "newsway" in url or "vop" in url:
                            continue
                    response = requests.get(url, headers={'User-agent': 'Mozila/5.0'})     
                    html = response.text                                                   
                    soup = BeautifulSoup(html, "html.parser")   
                    
                    if "entertain" in response.url:                                        
                        title = soup.select_one(".end_tit").text.strip()                                 
                        content = soup.select_one("#articeBody").text.strip()                           
                    elif "sports" in response.url:                                          
                        title = soup.select_one("h4.title").text.strip()                                 
                        content = soup.select_one("#newsEndContents")                    

                        divs = content.select("div")
                        for div in divs:
                            div.decompose()
                        paragraphs = content.select("p")
                        for p in paragraphs:
                            p.decompose()
                        content = content.text.strip()
                    else:
                        title = soup.select_one(".media_end_head_headline").text.strip()                 
                        content = soup.select_one("#dic_area").text.strip()

                else:
                    dsc_wrap = article.select_one("div.dsc_wrap")
                    dsc_thumb = dsc_wrap.select_one("a.api_txt_lines.dsc_txt_wrap")
                    if dsc_thumb != None : 
                        url = dsc_thumb.attrs["href"]
                        if "newsway" in url or "vop" in url:
                            continue
                        response = requests.get(url, headers={'User-agent': 'Mozila/5.0'})     
                        html = response.text
                        soup = BeautifulSoup(html, "html.parser")
                        
                        if links[0].attrs["href"]=="http://www.tvdaily.co.kr":
                            title = article.find("a", {"class":"news_tit"}).attrs["title"]
                            content = dsc_wrap.find("a").find("mark").text.strip()
                        elif "stoo" in url or "celuvmedia" in url or "bbsi" in url or "kwangju" in url:
                            1 == 1 #do nothing
                        else:
                            og_title = soup.find("meta", {"property":"og:title"})
                            if og_title!=None:
                                title = og_title.attrs["content"]
                            og_content = soup.find("meta", {"property":"og:description"})
                            if og_content != None:
                                content = og_content.attrs["content"]


                total_sheet[f"A{row+1}"] = row
                total_sheet[f"B{row+1}"] = url
                total_sheet[f"C{row+1}"] = title
                total_sheet[f"D{row+1}"] = content
                row += 1

            if keyword=="다.":
                wb.save(f"./resources/전체.xlsx")
            else:
                wb.save(f"./resources/{keyword}.xlsx")
        except:
            continue

    return row

In [33]:
lastpage = get_last_page_num("다.",7)
num = crawler("다.",lastpage,7)
print(f"--------------------- {num} articles have been crawled ---------------------\n\n") 

--------------------- 76 articles have been crawled ---------------------




# 2. KEYWORD EXTRACTOR

In [34]:
!pip install keybert
!pip install kiwipiepy
!pip install cython
from keybert import KeyBERT
from kiwipiepy import Kiwi
from transformers import BertModel
from openpyxl import load_workbook
import torch

model = BertModel.from_pretrained('skt/kobert-base-v1')
kw_model = KeyBERT(model)
kiwi = Kiwi()
keywords = []

def noun_extractor(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if len(token) != 1 and pos.startswith('N') or pos.startswith('SL'):
            results.append(token)
    return results

def keyword_extract(filename):
    load_wb = load_workbook(f"./resources/{filename}", data_only=True)
    load_ws = load_wb.get_sheet_by_name(filename.split('.')[0])
    articles = []
    for i in range(1, load_ws.max_row + 1):
        if load_ws.cell(row=i, column=4).value == None:
            articles.append("경찰청 창살 철창살은 녹슨 철창살인가 녹안슨 철창살인가")
        else: articles.append(load_ws.cell(row=i, column=4).value)

    noun_articles = []
    for article in articles:
        nouns = noun_extractor(article)
        article = ' '.join(nouns)
        noun_articles.append(article)

    n_gram = 2
    for i in range(1, load_ws.max_row+1):
        x = kw_model.extract_keywords(noun_articles[i-1], keyphrase_ngram_range=(1, n_gram), stop_words=stopwords, top_n=1)
        if x != []:
            load_ws.cell(row=i, column=5, value=f"{x[0]}")
        else:
            x = ("-",0)
            load_ws.cell(row=i, column=5, value=f"-")
        keywords.append(x[0])

    load_wb.save(f"./resources/{filename}")
    load_wb.close()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [35]:
keyword_extract("전체.xlsx")

/var/folders/k1/dqklgtzs47vckvrkl74mxsxc0000gn/T/ipykernel_1008/276851705.py:25: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  load_ws = load_wb.get_sheet_by_name(filename.split('.')[0])


# 3. GET TOP 30 KEYWORDS from whole articles

In [36]:
from collections import Counter

In [37]:
y = Counter(keywords).most_common(40)
top_keywords = []
i=0
for i in range(len(y)):
    if "기자" not in y[i][0][0] and "뉴스" not in y[i][0][0] and "경찰청 창살" not in y[i][0] and "-" not in y[i][0]:
        top_keywords.append(y[i][0][0])
print(f"===============        TOP Keywords        =================")
for i in range(len(top_keywords)):
    print(f"                  {i}.  {top_keywords[i]}")

===============        TOP Keywords        =================
                  0.  인스타그램 사진
                  1.  대변인 천안함
                  2.  김광현 벌금
                  3.  지지자 네티즌
                  4.  이상민 민주당
                  5.  위원장 사태
                  6.  패션 fastfashion
                  7.  스릴러 주역
                  8.  kbo 상벌
                  9.  mc 김종국
                  10.  월드컵 스타
                  11.  이하 월드컵
                  12.  미국 공군
                  13.  교회 베이비박스
                  14.  건물주 리모델링
                  15.  게스트 출연
                  16.  kbs joy
                  17.  최근 우크라이나
                  18.  한국 티타임
                  19.  학생 사고
                  20.  몬스타엑스 유튜브
                  21.  서태 정영주
                  22.  주차 기숙사
                  23.  엄마 이찬원
                  24.  배포 무료
                  25.  스포츠한국 조은애
                  26.  가정 아반떼
                  27.  김지은 인턴
                  28.  사진 채정안
                  29.  한국노총 경사노위
                  30.  

# 4. RECRAWL ARTICLES INCLUDING THE KETWORDS

In [38]:
zero_index = []
for i in range(len(top_keywords)):
    lastpage = get_last_page_num(top_keywords[i],9)
    num = crawler(top_keywords[i],lastpage,9)
    if num==0:
        zero_index.append(i)
    print(f"--------------------- {num} articles including the keyword [{top_keywords[i]}] have been crawled ---------------------\n\n") 
for i in zero_index:
    top_keywords.remove(top_keywords[i])

--------------------- 40 articles including the keyword [인스타그램 사진] have been crawled ---------------------


--------------------- 50 articles including the keyword [대변인 천안함] have been crawled ---------------------


--------------------- 10 articles including the keyword [김광현 벌금] have been crawled ---------------------


--------------------- 1 articles including the keyword [지지자 네티즌] have been crawled ---------------------


--------------------- 21 articles including the keyword [이상민 민주당] have been crawled ---------------------


--------------------- 69 articles including the keyword [위원장 사태] have been crawled ---------------------


--------------------- 1 articles including the keyword [패션 fastfashion] have been crawled ---------------------


--------------------- 2 articles including the keyword [스릴러 주역] have been crawled ---------------------


--------------------- 10 articles including the keyword [kbo 상벌] have been crawled ---------------------


--------------------- 2 art

# 5. GET SUMMARY & TOPIC from EACH KEYWORD FILES

## (1) Summary

In [39]:
!pip install torch
!pip install transformers

import torch
from transformers import PreTrainedTokenizerFast
from transformers.models.bart import BartForConditionalGeneration
import pandas as pd
import re
import json


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [40]:
class summaryModel:
  
  def __init__(self):
    self.model = BartForConditionalGeneration.from_pretrained("ainize/kobart-news") #kobart 모델 사용
    self.tokenizer = PreTrainedTokenizerFast.from_pretrained('ainize/kobart-news')

  def summarize(self, text):
    input_ids = self.tokenizer.encode(text, return_tensors="pt")
    #input_ids = input_ids.unsqueeze(0)
    output = self.model.generate(input_ids, eos_token_id=1, max_length=128, num_beams=5)
    output = self.tokenizer.decode(output[0], skip_special_tokens=True)
    return self.postprocess(output)
  
  def postprocess(self, text): #문장이 중간에 끊기지 않도록 후처리
    text = text.split(".")[:-1]
    out_text = [t for t in text if len(t)>5]
    return ". ".join(out_text)
    
sum = summaryModel()

def summarization_article(keyword):
  data = pd.read_excel(f'./resources/{keyword}.xlsx', header=None, sheet_name = keyword, index_col = 0, names = ["index", "url", "title", "content"]) 

  #데이터전처리
  data['drop'] = data['content'].apply(lambda x : 1 if type(x) != str else 0)
  data = data.drop(data[data['drop']==1].index)
  data["content"] = data["content"].apply(lambda x:  re.sub(r'\([^)]*\)', '', x))
  data["content"] = data["content"].apply(lambda x:  re.sub(r"[^가-힇0-9A-Za-z‘“.,!?\n/ ]+", " ", x))
  data["content"] = data["content"].apply(lambda x:  re.sub(r"[(\n)]+","\n",x))
  data = data.dropna()


  #각 기사에 대한 요약 저장
  content = data["content"].tolist()
  sum_text = []
  for c in content:
    if type(c)==str and len(c)>=1:
      x = c.split(" ")
      sum_text.append(sum.summarize(" ".join(x[:500])))

  #각 summary로 한번 더 요약을 한 버전
  sum_text = ". ".join(sum_text).split(" ")[:500]
  output = sum.summarize(" ".join(sum_text))
  return output

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [41]:
links = []
summaries =[]

passed_keyword = []
for key in top_keywords:
    try:
        summaries.append(summarization_article(key))
        wb = load_workbook(f"./resources/{key}.xlsx")
        ws = wb[key]
        links.append(ws.cell(row=1, column=2).value) 
    except IndexError:
        passed_keyword.append(key)
        continue
    except FileNotFoundError:
        passed_keyword.append(key)
        continue
    if summaries[-1] == "":
        summaries[-1] = (ws.cell(row=1, column=3).value).replace(".","")
for i in passed_keyword:
    top_keywords.remove(i)

KeyboardInterrupt: 

## (2) Topic

In [42]:
!pip install transformers
!pip install --upgrade jupyter
!pip install --upgrade ipywidgets

import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm

import pandas as pd
import transformers
from transformers import AutoTokenizer, AdamW, RobertaForSequenceClassification
from ipywidgets import IntProgress
from collections import Counter
from openpyxl import load_workbook


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [43]:
topics = []

def topic_extract(keyword):
  try:
    load_wb = load_workbook("./resources/"+keyword + ".xlsx", data_only=True)
    load_ws = load_wb.get_sheet_by_name(keyword)
    test_data = open("test_data.csv", "w", encoding="utf-8-sig")
    test_data.write("index,topic\n")
    for i in range(1, load_ws.max_row + 1):
        if (load_ws.cell(row=i, column=3).value == None):
            test_data.write(str(load_ws.cell(row=i, column=1).value) + ',' + "UNK\n")
        else:
            test_data.write(str(load_ws.cell(row=i, column=1).value) + ',' + load_ws.cell(row=i, column=3).value.replace(",", "").replace(".","").replace("[","").replace("]","").replace("(","").replace(")","").replace('"',"").replace("'","").replace(":","").replace("-","").replace("·","") + "\n")
    test_data.close()
    test = pd.read_csv('test_data.csv', encoding="utf-8-sig")
    class NTDataset_test(Dataset):
      
      def __init__(self, csv_file):
        self.dataset = csv_file
        self.tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

        print(self.dataset.describe())
      
      def __len__(self):
        return len(self.dataset)
      
      def __getitem__(self, idx):
        row = self.dataset.iloc[idx, 1:2].values
        text = row[0]
        inputs = self.tokenizer(
            text, 
            return_tensors='pt',
            truncation=True,
            max_length=15,
            pad_to_max_length=True,
            add_special_tokens=True
            )
        
        input_ids = inputs['input_ids'][0]
        attention_mask = inputs['attention_mask'][0]

        return input_ids, attention_mask
    batch_size = 128
    test_dataset = NTDataset_test(test)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    if torch.backends.mps.is_available:
        device = torch.device("mps")
    elif torch.cuda.is_available:
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")
    # test
    model = RobertaForSequenceClassification.from_pretrained("klue/roberta-large", num_labels=7).to(device)
    model.load_state_dict(torch.load("model.pth"))
    model.eval()

    pred = []

    for input_ids_batch, attention_masks_batch in tqdm(test_loader):
      y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
      _, predicted = torch.max(y_pred, 1)
      pred.extend(predicted.tolist())
    topic_list = ["IT과학", "경제", "사회", "생활문화", "세계", "스포츠", "정치"]
    temp = []
    for i in range(1, len(pred) + 1):
      temp.append(topic_list[pred[i - 1]])
    topics.append(Counter(temp).most_common(1)[0][0])
    # load_wb.save(filename)
    load_wb.close()
  except FileNotFoundError:
     pass
     


In [44]:
for key in top_keywords:
   try:
      topic_extract(key)
   except:
      

/var/folders/k1/dqklgtzs47vckvrkl74mxsxc0000gn/T/ipykernel_1008/3765010246.py:5: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  load_ws = load_wb.get_sheet_by_name(keyword)


           index
count  40.000000
mean   19.500000
std    11.690452
min     0.000000
25%     9.750000
50%    19.500000
75%    29.250000
max    39.000000


Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'class

  0%|          | 0/1 [00:00<?, ?it/s]

/var/folders/k1/dqklgtzs47vckvrkl74mxsxc0000gn/T/ipykernel_1008/3765010246.py:5: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  load_ws = load_wb.get_sheet_by_name(keyword)


...

/var/folders/k1/dqklgtzs47vckvrkl74mxsxc0000gn/T/ipykernel_1008/3765010246.py:5: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  load_ws = load_wb.get_sheet_by_name(keyword)


          index
count  6.000000
mean   2.500000
std    1.870829
min    0.000000
25%    1.250000
50%    2.500000
75%    3.750000
max    5.000000


Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'class

  0%|          | 0/1 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: './resources/스포츠한국 조은애.xlsx'

# 6. SEND DATA(json) to SERVER

In [ ]:
import json

with open("data.json", 'r', encoding="utf-8-sig") as f:
    myJson = {}
    myJson["keywords"] = top_keywords
    myJson["topics"] = topics
    myJson["links"] = links
    myJson["summaries"] = summaries

with open("/Users/eugene/Downloads/startbootstrap-creative-gh-pages/data.json", 'w+', encoding="utf-8-sig") as f:
    json.dump(myJson, f, ensure_ascii=False)


In [ ]:

stopwords = """
아
휴
아이구
아이쿠
아이고
어
나
우리
저희
따라
의해
을
를
에
의
가
으로
로
에게
뿐이다
의거하여
근거하여
입각하여
기준으로
예하면
예를 들면
예를 들자면
저
소인
소생
저희
지말고
하지마
하지마라
다른
물론
또한
그리고
비길수 없다
해서는 안된다
뿐만 아니라
만이 아니다
만은 아니다
막론하고
관계없이
그치지 않다
그러나
그런데
하지만
든간에
논하지 않다
따지지 않다
설사
비록
더라도
아니면
만 못하다
하는 편이 낫다
불문하고
향하여
향해서
향하다
쪽으로
틈타
이용하여
타다
오르다
제외하고
이 외에
이 밖에
하여야
비로소
한다면 몰라도
외에도
이곳
여기
부터
기점으로
따라서
할 생각이다
하려고하다
이리하여
그리하여
그렇게 함으로써
하지만
일때
할때
앞에서
중에서
보는데서
으로써
로써
까지
해야한다
일것이다
반드시
할줄알다
할수있다
할수있어
임에 틀림없다
한다면
등
등등
제
겨우
단지
다만
할뿐
딩동
댕그
대해서
대하여
대하면
훨씬
얼마나
얼마만큼
얼마큼
남짓
여
얼마간
약간
다소
좀
조금
다수
몇
얼마
지만
하물며
또한
그러나
그렇지만
하지만
이외에도
대해 말하자면
뿐이다
다음에
반대로
반대로 말하자면
이와 반대로
바꾸어서 말하면
바꾸어서 한다면
만약
그렇지않으면
까악
툭
딱
삐걱거리다
보드득
비걱거리다
꽈당
응당
해야한다
에 가서
각
각각
여러분
각종
각자
제각기
하도록하다
와
과
그러므로
그래서
고로
한 까닭에
하기 때문에
거니와
이지만
대하여
관하여
관한
과연
실로
아니나다를가
생각한대로
진짜로
한적이있다
하곤하였다
하
하하
허허
아하
거바
와
오
왜
어째서
무엇때문에
어찌
하겠는가
무슨
어디
어느곳
더군다나
하물며
더욱이는
어느때
언제
야
이봐
어이
여보시오
흐흐
흥
휴
헉헉
헐떡헐떡
영차
여차
어기여차
끙끙
아야
앗
아야
콸콸
졸졸
좍좍
뚝뚝
주룩주룩
솨
우르르
그래도
또
그리고
바꾸어말하면
바꾸어말하자면
혹은
혹시
답다
및
그에 따르는
때가 되어
즉
지든지
설령
가령
하더라도
할지라도
일지라도
지든지
몇
거의
하마터면
인젠
이젠
된바에야
된이상
만큼	어찌됏든
그위에
게다가
점에서 보아
비추어 보아
고려하면
하게될것이다
일것이다
비교적
좀
보다더
비하면
시키다
하게하다
할만하다
의해서
연이서
이어서
잇따라
뒤따라
뒤이어
결국
의지하여
기대여
통하여
자마자
더욱더
불구하고
얼마든지
마음대로
주저하지 않고
곧
즉시
바로
당장
하자마자
밖에 안된다
하면된다
그래
그렇지
요컨대
다시 말하자면
바꿔 말하면
즉
구체적으로
말하자면
시작하여
시초에
이상
허
헉
허걱
바와같이
해도좋다
해도된다
게다가
더구나
하물며
와르르
팍
퍽
펄렁
동안
이래
하고있었다
이었다
에서
로부터
까지
예하면
했어요
해요
함께
같이
더불어
마저
마저도
양자
모두
습니다
가까스로
하려고하다
즈음하여
다른
다른 방면으로
해봐요
습니까
했어요
말할것도 없고
무릎쓰고
개의치않고
하는것만 못하다
하는것이 낫다
매
매번
들
모
어느것
어느
로써
갖고말하자면
어디
어느쪽
어느것
어느해
어느 년도
라 해도
언젠가
어떤것
어느것
저기
저쪽
저것
그때
그럼
그러면
요만한걸
그래
그때
저것만큼
그저
이르기까지
할 줄 안다
할 힘이 있다
너
너희
당신
어찌
설마
차라리
할지언정
할지라도
할망정
할지언정
구토하다
게우다
토하다
메쓰겁다
옆사람
퉤
쳇
의거하여
근거하여
의해
따라
힘입어
그
다음
버금
두번째로
기타
첫번째로
나머지는
그중에서
견지에서
형식으로 쓰여
입장에서
위해서
단지
의해되다
하도록시키다
뿐만아니라
반대로
전후
전자
앞의것
잠시
잠깐
하면서
그렇지만
다음에
그러한즉
그런즉
남들
아무거나
어찌하든지
같다
비슷하다
예컨대
이럴정도로
어떻게
만약
만일
위에서 서술한바와같이
인 듯하다
하지 않는다면
만약에
무엇
무슨
어느
어떤
아래윗
조차
한데
그럼에도 불구하고
여전히
심지어
까지도
조차도
하지 않도록
않기 위하여
때
시각
무렵
시간
동안
어때
어떠한
하여금
네
예
우선
누구
누가 알겠는가
아무도
줄은모른다
줄은 몰랏다
하는 김에
겸사겸사
하는바
그런 까닭에
한 이유는
그러니
그러니까
때문에
그
너희
그들
너희들
타인
것
것들
너
위하여
공동으로
동시에
하기 위하여
어찌하여
무엇때문에
붕붕
윙윙
나
우리
엉엉
휘익
윙윙
오호
아하
어쨋든
만 못하다	하기보다는
차라리
하는 편이 낫다
흐흐
놀라다
상대적으로 말하자면
마치
아니라면
쉿
그렇지 않으면
그렇지 않다면
안 그러면
아니었다면
하든지
아니면
이라면
좋아
알았어
하는것도
그만이다
어쩔수 없다
하나
일
일반적으로
일단
한켠으로는
오자마자
이렇게되면
이와같다면
전부
한마디
한항목
근거로
하기에
아울러
하지 않도록
않기 위해서
이르기까지
이 되다
로 인하여
까닭으로
이유만으로
이로 인하여
그래서
이 때문에
그러므로
그런 까닭에
알 수 있다
결론을 낼 수 있다
으로 인하여
있다
어떤것
관계가 있다
관련이 있다
연관되다
어떤것들
에 대해
이리하여
그리하여
여부
하기보다는
하느니
하면 할수록
운운
이러이러하다
하구나
하도다
다시말하면
다음으로
에 있다
에 달려 있다
우리
우리들
오히려
하기는한데
어떻게
어떻해
어찌됏어
어때
어째서
본대로
자
이
이쪽
여기
이것
이번
이렇게말하자면
이런
이러한
이와 같은
요만큼
요만한 것
얼마 안 되는 것
이만큼
이 정도의
이렇게 많은 것
이와 같다
이때
이렇구나
것과 같이
끼익
삐걱
따위
와 같은 사람들
부류의 사람들
왜냐하면
중의하나
오직
오로지
에 한하다
하기만 하면
도착하다
까지 미치다
도달하다
정도에 이르다
할 지경이다
결과에 이르다
관해서는
여러분
하고 있다
한 후
혼자
자기
자기집
자신
우에 종합한것과같이
총적으로 보면
총적으로 말하면
총적으로
대로 하다
으로서
참
그만이다
할 따름이다
쿵
탕탕
쾅쾅
둥둥
봐
봐라
아이야
아니
와아
응
아이
참나
년
월
일
령
영
일
이
삼
사
오
육
륙
칠
팔
구
이천육
이천칠
이천팔
이천구
하나
둘
셋
넷
다섯
여섯
일곱
여덟
아홉
령
영
""".split()